# Reading PHYSLITE files using the `PHYSLITESchema`

`NanoEvents` is a `coffea` utility to wrap NTuple structures (such as the ATLAS PHYSLITE format) into a single awkward array with appropriate object methods (such as Lorentz vector methods), cross references, and nested objects. All are by default accessed in delayed* mode from the input ROOT TTree via `uproot`.  

The interpretation of the TTree data is configurable via schema objects, which are community-supplied for various input file types. Relevant schemas for this training:
1. `BaseSchema`, which provides a simple representation of the input TTree, where each branch is available verbatim as `events.branch_name`. Any branches that `uproot` supports at "full speed" (i.e. that are fully split and either flat or single-jagged) can be read by this schema.
2. `PHYSLITESchema`, for the ATLAS PHYSLITE derivations.

The schema code, along with their specialized methods, can be found here: https://github.com/CoffeaTeam/coffea/tree/master/src/coffea/nanoevents

\* _delayed_ access refers to only fetching the needed data from the file when you're ready to compute the needed quantities. Until then, a minimal amount of metadata is used to trace the operations to eventually be performed (frequently called "lazy" evaluation).

In this demo we will read the content of a PHYSLITE file using the `PHYSLITESchema`. The usage and benefits of the schema will be demonstrated.

In [ ]:
import coffea
print("coffea version: ", coffea.__version__)
import awkward as ak
print("awkward version: ", ak.__version__)
from coffea.nanoevents import NanoEventsFactory, PHYSLITESchema
import numpy as np
import matplotlib.pyplot as plt

Let's load a PHYSLITE file containing simulated event data describing the production the Higgs boson and its decay via a pair of Z bosons to a four lepton final state. Those data are part of the recent ATLAS open data release and can be found at DOI:[10.7483/OPENDATA.ATLAS.Z2J9.709J](http://doi.org/10.7483/OPENDATA.ATLAS.Z2J9.709J).

More information about our data release at [this article](https://atlas.cern/Updates/News/Open-Data-Research) and the ATLAS open data portal: https://opendata.atlas.cern/

You can download those files at your local machine or you can stream them directly.

In [ ]:
# Hgg -> 4l sample

# local
# file_path = '/Users/ekourlitis/cernbox/mc20_13TeV.345060.PowhegPythia8EvtGen_NNLOPS_nnlo_30_ggH125_ZZ4l.deriv.DAOD_PHYSLITE.e7735_s3681_r13167_p6026/mc20_13TeV/DAOD_PHYSLITE.38191712._000020.pool.root.1'

# stream
file_path = "root://eospublic.cern.ch//eos/opendata/atlas/rucio/mc20_13TeV/DAOD_PHYSLITE.38191712._000001.pool.root.1"

To make the loading lighter, we will use the `filter_name` function, in order to load only the variables needed. Then we will use `coffea`'s `NanoEventsFactory.from_root` to load the file. Consider looking at the [`from_root`](https://coffeateam.github.io/coffea/api/coffea.nanoevents.NanoEventsFactory.html#coffea.nanoevents.NanoEventsFactory.from_root) class method to see all optional arguments.

All the variables stored in the open data PHYSLITE files are listed in: https://atlas-physlite-content-opendata.web.cern.ch/. For your reference, the equivalent internal webpage for general PHYSLITE files is: https://atlas-physlite-content.web.cern.ch/.

_Some warnings will appear as we're still working with `uproot` and the `PHYSLITESchema` to correctly interpret all the variables stored in PHYSLITE._

In [ ]:
# load in memory

def filter_name(name):
    '''
    Load only the variables needed.
    '''
    return name in [
        "EventInfoAuxDyn.mcEventWeights",
        
        "AnalysisElectronsAuxDyn.pt",
        "AnalysisElectronsAuxDyn.eta",
        "AnalysisElectronsAuxDyn.phi",
        "AnalysisElectronsAuxDyn.m",
        
        "AnalysisMuonsAuxDyn.pt",
        "AnalysisMuonsAuxDyn.eta",
        "AnalysisMuonsAuxDyn.phi",
        "AnalysisMuonsAuxDyn.m",
        
        "AnalysisJetsAuxDyn.pt",
        "AnalysisJetsAuxDyn.eta",
        "AnalysisJetsAuxDyn.phi",
        "AnalysisJetsAuxDyn.m",
        "AnalysisJetsAuxDyn.btaggingLink",
        
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pb",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pc",
        "BTagging_AntiKt4EMPFlowAuxDyn.DL1dv01_pu",
    ]

events = NanoEventsFactory.from_root(
    {file_path: "CollectionTree"}, # all the event variables are stored in the TTree called CollectionTree
    schemaclass=PHYSLITESchema, # tell NanoEventsFactory.from_root that you read a PHYSLITE file
    uproot_options=dict(filter_name=filter_name)
).events()

The `events` object is an awkward array, which at its top level is a record array with one record for each "collection" (or "container"), where a collection is a grouping of variables (TBranches) based on the naming conventions of `PHYSLITESchema`. For example, in the file we opened there are the collections:

In [ ]:
events.fields

For each collection, we can list the variables that have been loaded. For example, let's look at the variables of the `Electrons`:

In [ ]:
events.Electrons.fields

The data in a PHYSLITE file are highly structured but jagged. In other words, in each event there are different number of objects in each collection, thus the awkward arrays are not rectilinear but _awkward_. We can see that if we actually request to compute the objects of the `Electrons` collection.

_This is the first time we will call the `compute()` method. Actual number crunching is happening here!_

In [ ]:
events.Electrons.compute()

## Slicing

One of the most common operations you will found yourself doing is event selection or array _slicing_. We can practice this by first defining an awkward (boolean) array by a logical operation. Then we can use this array as an argument to slice our events.

For example, let's request events with at least two electrons:

In [ ]:
# define boolean array
selection_2e = ak.num(events.Electrons, axis=-1) > 1

# print the selected events
events[selection_2e].compute()

Notice how the number of events in the awkward array has been reduced.

In [ ]:
# print the second of the selected events
events[selection_2e][1].compute()

In [ ]:
# print the leading electron pt of the selected events
events.Electrons[selection_2e][:, 0].pt.compute() # in MeV
# events[selection_2e][:, "Electrons"][:, "pt"][:, 0].compute() # another uglier way to write this

Next, we will demonstrate something more complicated. This is an example of the power of the schemas along with vector operations*. We will use a utility method [`delta_r`](https://coffeateam.github.io/coffea/api/coffea.nanoevents.methods.vector.LorentzVector.html#coffea.nanoevents.methods.vector.LorentzVector.delta_r) to calculate the distance $\Delta R = \sqrt{\Delta\eta^2+\Delta\phi^2}$ between two LorentzVector objects. In particular, we will calculate the distance between the leading and sub-leading electrons in each event. Of course, we need to make use we use events that have at least two electrons, thus we will utilize the `selection_2e` from above.

To plot a histogram we will just use `matplotlib` for the moment. You might also want to try the [`hist`](https://github.com/scikit-hep/hist) library which you will learn about this afternoon:

https://indico.cern.ch/event/1376945/timetable/?view=standard#33-introduction-to-hist

_\* `coffea` still uses the internal [`coffea.nanoevents.methods.vector`](https://coffeateam.github.io/coffea/modules/coffea.nanoevents.methods.vector.html) but it will soon switch to the [`vector`](https://github.com/scikit-hep/vector) library, which will hopefully be feature-compatible._

In [ ]:
# distance between leading and sub-leading electron in every event
dr = events.Electrons[selection_2e][:, 0].delta_r(events.Electrons[selection_2e][:, 1])

# compute and plot
plt.hist(dr.compute(), bins=50, range=(0, 5))
plt.xlabel(r"$\Delta R(e_0, e_1)$")
plt.ylabel("Events")
plt.show()

Let's now use the schema to calculate the invariant mass of a group of particles. In particular, calculate and plot the invariant mass of the four leading electrons of each event.

<details>
<summary><b>Hint</b></summary>

Use the [mass](https://coffeateam.github.io/coffea/api/coffea.nanoevents.methods.vector.LorentzVector.html#coffea.nanoevents.methods.vector.LorentzVector.mass) attribute.
</details>

In [ ]:
# your code here

<details>
<summary><b>Answer</b></summary>

```python
# define boolean array to select events with 4 electrons
selection_4e = ak.num(events.Electrons, axis=-1) == 4

# calculate the invariant mass of the 4 electrons
mass = (events.Electrons[selection_4e][:, 0] + 
        events.Electrons[selection_4e][:, 1] + 
        events.Electrons[selection_4e][:, 2] + 
        events.Electrons[selection_4e][:, 3]).mass

# compute and plot
plt.hist((mass/1000).compute(), bins=10, range=(100, 150))
plt.xlabel(r"$m(4e)$ [GeV]")
plt.ylabel("Events")
plt.show()
```
</details>

## Element Links

Some collections contain information which is contextually relevant to another collection. For example, in PHYSLITE files we store the `BTagging_AntiKt4EMPFlow` collection which contains information regarding the probabilities of the flavour origin of the objects of the `Jets` collection.

The information is referenced and accessed by the `ElementLinks`. `ElementLinks` are like pointers that help point to specific objects in a collection. They have a structure similar to a Python dictionary with a key (`m_persKey`) to identify the collection they point to and a value (`m_persIndex`) to identify the index of the element in that collection. 

On the above example, each object of the `Jets` collection (i.e. each jet) holds a variable called `btaggingLink`, which points to the objects of the `BTagging_AntiKt4EMPFlow` collection. The index is used to identify which exactly `BTagging_AntiKt4EMPFlow` object corresponds to each jet.

In [ ]:
events.Jets.btaggingLink.fields

### Naive Linking

We're currently working with `coffea` and the `PHYSLITESchema` to enable automatic cross-reference (_ElementLinking_) between collections. However, in simple cases we can achieve the linking, without actually using the `ElementLinks`.

When there is **one-to-one** correspondence between the objects of two collections, we can link those without the need to resolve the actual links. For our example, each object of the `Jets` collection is linked to one object of the `BTagging_AntiKt4EMPFlow` collection.

We will write a function to perform that linking. We will use the [DL1dv01](https://ftag.docs.cern.ch/algorithms/taggers/dl1/) flavour-tagging algorithm and information about its calibration can be found [here](https://ftag.docs.cern.ch/recommendations/algs/r22-preliminary/#working-point-definition-for-dl1dv01).

In [ ]:
def calculate_jets_DL1dv01(events):
    
    BTagging = events.BTagging_AntiKt4EMPFlow
    
    f_c = 0.018
    DL1dv01 = BTagging.DL1dv01_pb/(f_c*BTagging.DL1dv01_pc + (1-f_c)*BTagging.DL1dv01_pu)
    DL1dv01 = np.log(DL1dv01)

    return DL1dv01

We will use the above function to assign a new variable to the `Jets` collection. One can assign new variables to the arrays, with some caveats:

* Assignment must use `events["path", "to", "name"] = value`.
* Assignment to a sliced events won't be accessible from the original variable.

In [ ]:
# assign new variable to the collection
events['Jets', 'DL1dv01'] = calculate_jets_DL1dv01(events)

events.Jets.fields

Let's finally use the new variable to make a selection. We will plot the leading jet transverse momentum ($p_T$) for events that have at least one b-tagged jet at the 77% efficiency working point.

In [ ]:
# for events at least one b-jets at 77% WP
selection_1bjet = ak.sum(events.Jets.DL1dv01 > 2.456, axis=-1) > 0

# plot the leading jet pt
plt.hist((events.Jets[selection_1bjet][:, 0].pt/1000).compute(), bins=50, range=(0, 500))
plt.xlabel(r"$p_T(j_0)$ [GeV]")
plt.ylabel("Events")
plt.show()